In [1]:
import os
import json
import pandas as pd
from components import db_functions

In [3]:
df = pd.read_csv(os.path.join('data','db','db build files','uniprotkb_AND_reviewed_true_2023_09_04.tsv'),sep='\t')

In [19]:
map = {'Entry': 'accession',
            'Post-translational modification': 'cc_ptm',
            'Comment Count': 'comment_count',
            'Organism': 'organism_name',
            'Modified residue': 'ft_mod_res',
            'Gene names': 'gene_names',
            'Gene Names': 'gene_names',
            'Gene names (primary)': 'gene_primary',
            'Gene Names (primary)': 'gene_primary',
            'Gene ontology (cellular component)': 'go_c',
            'Gene ontology (molecular function)': 'go_f',
            'Gene ontology (biological process)': 'go_p',
            'Entry name': 'id',
            'Entry Name': 'id',
            'Keywords': 'keyword',
            'Length': 'length',
            'PubMed ID': 'lit_pubmed_id',
            'Mass': 'mass',
            'Protein names': 'protein_name',
            'Reviewed': 'reviewed',
            'Sequence': 'sequence',
            'Entry version': 'version',
            'Reactome': 'xref_reactome',
            'Cofactor': 'cc_cofactor',
            'Developmental stage': 'cc_developmental_stage',
            'Involvement in disease': 'cc_disease',
            'Disruption phenotype': 'cc_disruption_phenotype',
            'Domain[CC]': 'cc_domain',
            'Function [CC]': 'cc_function',
            'Induction': 'cc_induction',
            'Interacts with': 'cc_interaction',
            'Miscellaneous [CC]': 'cc_miscellaneous',
            'Sequence caution': 'cc_sequence_caution',
            'Subcellular location[CC]': 'cc_subcellular_location',
            'Date of last sequence modification': 'date_sequence_modified',
            'Organism ID': 'organism_id',
            'PhosphoSitePlus': 'xref_phosphositeplus',
            'Sequence version': 'sequence_version',
            'RefSeq': 'xref_refseq',
            'STRING': 'xref_string',
            'Ensembl': 'xref_ensembl',
            'IntAct': 'xref_intact',
            'InterPro': 'xref_interpro',
            'PANTHER': 'xref_panther',
            'SUPFAM': 'xref_supfam',
            'Pfam': 'xref_pfam',
            'PRO': 'xref_pro',
            'PROSITE': 'xref_prosite',
            'Alternative sequence': 'ft_var_seq',
            'Alternative products (isoforms)': 'cc_alternative_products'
            }
print(','.join([map[c] if c in map else f'{c} not in map' for c in df.columns]))

accession,reviewed,gene_primary,id,gene_names,organism_name,length,cc_alternative_products,ft_var_seq,sequence


In [36]:
fils = {'Entry': 'accession',
            'Absorption': 'absorption',
            'Annotation': 'annotation_score',
            'Activity regulation': 'cc_activity_regulation',
            'Allergenic properties': 'cc_allergen',
            'Alternative products': 'cc_alternative_products',
            'Biotechnological use': 'cc_biotechnology',
            'Catalytic activity': 'cc_catalytic_activity',
            'Caution': 'cc_caution',
            'Cofactor': 'cc_cofactor',
            'Developmental stage': 'cc_developmental_stage',
            'Involvement in disease': 'cc_disease',
            'Disruption phenotype': 'cc_disruption_phenotype',
            'Domain[CC]': 'cc_domain',
            'Function [CC]': 'cc_function',
            'Induction': 'cc_induction',
            'Interacts with': 'cc_interaction',
            'Mass spectrometry': 'cc_mass_spectrometry',
            'Miscellaneous [CC]': 'cc_miscellaneous',
            'Pathway': 'cc_pathway',
            'Pharmaceutical use': 'cc_pharmaceutical',
            'Polymorphism': 'cc_polymorphism',
            'Post-translational modification': 'cc_ptm',
            'RNA editing': 'cc_rna_editing',
            'Sequence caution': 'cc_sequence_caution',
            'Subcellular location[CC]': 'cc_subcellular_location',
            'Subunit structure[CC]': 'cc_subunit',
            'Tissue specificity': 'cc_tissue_specificity',
            'Toxic dose': 'cc_toxic_dose',
            'Comment Count': 'comment_count',
            'Date of creation': 'date_created',
            'Date of last modification': 'date_modified',
            'Date of last sequence modification': 'date_sequence_modified',
            'EC number': 'ec',
            'Erroneous gene model prediction': 'error_gmodel_pred',
            'Features': 'feature_count',
            'Fragment': 'fragment',
            'Active site': 'ft_act_site',
            'Binding site': 'ft_binding',
            # this field name causes problems 2022-08-22
            # 'Calcium binding': 'ft_ca_bind',
            'Glycosylation': 'ft_carbohyd',
            'Chain': 'ft_chain',
            'Coiled coil': 'ft_coiled',
            'Compositional bias': 'ft_compbias',
            'Sequence conflict': 'ft_conflict',
            'Cross-link': 'ft_crosslnk',
            'Disulfide bond': 'ft_disulfid',
            'DNA binding': 'ft_dna_bind',
            'Domain[FT]': 'ft_domain',
            'Helix': 'ft_helix',
            'Initiator methionine': 'ft_init_met',
            'Intramembrane': 'ft_intramem',
            'Lipidation': 'ft_lipid',
            # this field name causes problems 2022-08-22
            # 'Metal binding': 'ft_metal',
            'Modified residue': 'ft_mod_res',
            'Motif': 'ft_motif',
            'Mutagenesis': 'ft_mutagen',
            'Non-adjacent residues': 'ft_non_cons',
            'Non-standard residue': 'ft_non_std',
            'Non-terminal residue': 'ft_non_ter',
            # this field name causes problems 2022-08-22
            # 'Nucleotide binding': 'ft_np_bind',
            'Peptide': 'ft_peptide',
            'Propeptide': 'ft_propep',
            'Region': 'ft_region',
            'Repeat': 'ft_repeat',
            'Signal peptide': 'ft_signal',
            'Site': 'ft_site',
            'Beta strand': 'ft_strand',
            'Topological domain': 'ft_topo_dom',
            'Transit peptide': 'ft_transit',
            'Transmembrane': 'ft_transmem',
            'Turn': 'ft_turn',
            'Sequence uncertainty': 'ft_unsure',
            'Alternative sequence': 'ft_var_seq',
            'Natural variant': 'ft_variant',
            'Zinc finger': 'ft_zn_fing',
            'Gene names': 'gene_names',
            'Gene names (ordered locus)': 'gene_oln',
            'Gene names (ORF)': 'gene_orf',
            'Gene names (primary)': 'gene_primary',
            'Gene names (synonym)': 'gene_synonym',
            'Gene ontology (GO)': 'go',
            'Gene ontology (cellular component)': 'go_c',
            'Gene ontology (molecular function)': 'go_f',
            'Gene ontology IDs': 'go_id',
            'Gene ontology (biological process)': 'go_p',
            'Entry name': 'id',
            'Keywords': 'keyword',
            'Keyword ID': 'keywordid',
            'Kinetics': 'kinetics',
            'Length': 'length',
            'Taxonomic lineage': 'lineage',
            'Taxonomic lineage (IDs)': 'lineage_ids',
            'PubMed ID': 'lit_pubmed_id',
            'Mass': 'mass',
            'Gene encoded by': 'organelle',
            'Organism ID': 'organism_id',
            'Organism': 'organism_name',
            'pH dependence': 'ph_dependence',
            'Protein existence': 'protein_existence',
            'Protein families': 'protein_families',
            'Protein names': 'protein_name',
            'Redox potential': 'redox_potential',
            'Reviewed': 'reviewed',
            'Rhea ID': 'rhea',
            'Sequence': 'sequence',
            'Sequence version': 'sequence_version',
            '3D': 'structure_3d',
            'Temperature dependence': 'temp_dependence',
            'Tools': 'tools',
            'UniParc': 'uniparc_id',
            'Entry version': 'version',
            'Virus hosts': 'virus_hosts',
            'ABCD': 'xref_abcd',
            'Allergome': 'xref_allergome',
            'AlphaFoldDB': 'xref_alphafolddb',
            'Antibodypedia': 'xref_antibodypedia',
            'ArachnoServer': 'xref_arachnoserver',
            'Araport': 'xref_araport',
            'Bgee': 'xref_bgee',
            'BindingDB': 'xref_bindingdb',
            'BioCyc': 'xref_biocyc',
            'BioGRID': 'xref_biogrid',
            'BioGRID-ORCS': 'xref_biogrid-orcs',
            'BioMuta': 'xref_biomuta',
            'BMRB': 'xref_bmrb',
            'BRENDA': 'xref_brenda',
            'CarbonylDB': 'xref_carbonyldb',
            'CAZy': 'xref_cazy',
            'CCDS': 'xref_ccds',
            'CDD': 'xref_cdd',
            'CGD': 'xref_cgd',
            'ChEMBL': 'xref_chembl',
            'ChiTaRS': 'xref_chitars',
            'CLAE': 'xref_clae',
            'CleanEx': 'xref_cleanex',
            'CollecTF': 'xref_collectf',
            'ComplexPortal': 'xref_complexportal',
            'COMPLUYEAST-2DPAGE': 'xref_compluyeast-2dpage',
            'ConoServer': 'xref_conoserver',
            'CORUM': 'xref_corum',
            'CPTAC': 'xref_cptac',
            'CPTC': 'xref_cptc',
            'CTD': 'xref_ctd',
            'dbSNP': 'xref_dbsnp',
            'DEPOD': 'xref_depod',
            'dictyBase': 'xref_dictybase',
            'DIP': 'xref_dip',
            'DisGeNET': 'xref_disgenet',
            'DisProt': 'xref_disprot',
            'DMDM': 'xref_dmdm',
            'DNASU': 'xref_dnasu',
            'DOSAC-COBS-2DPAGE': 'xref_dosac-cobs-2dpage',
            'DrugBank': 'xref_drugbank',
            'DrugCentral': 'xref_drugcentral',
            'EchoBASE': 'xref_echobase',
            'eggNOG': 'xref_eggnog',
            'ELM': 'xref_elm',
            'EMBL': 'xref_embl',
            'Ensembl': 'xref_ensembl',
            'EnsemblBacteria': 'xref_ensemblbacteria',
            'EnsemblFungi': 'xref_ensemblfungi',
            'EnsemblMetazoa': 'xref_ensemblmetazoa',
            'EnsemblPlants': 'xref_ensemblplants',
            'EnsemblProtists': 'xref_ensemblprotists',
            'EPD': 'xref_epd',
            'ESTHER': 'xref_esther',
            'euHCVdb': 'xref_euhcvdb',
            'EvolutionaryTrace': 'xref_evolutionarytrace',
            'ExpressionAtlas': 'xref_expressionatlas',
            'FlyBase': 'xref_flybase',
            'Gene3D': 'xref_gene3d',
            'GeneCards': 'xref_genecards',
            'GeneID': 'xref_geneid',
            'GeneReviews': 'xref_genereviews',
            'GeneTree': 'xref_genetree',
            'Genevisible': 'xref_genevisible',
            'GeneWiki': 'xref_genewiki',
            'GenomeRNAi': 'xref_genomernai',
            'GlyConnect': 'xref_glyconnect',
            'GlyGen': 'xref_glygen',
            'Gramene': 'xref_gramene',
            'GuidetoPHARMACOLOGY': 'xref_guidetopharmacology',
            'HAMAP': 'xref_hamap',
            'HGNC': 'xref_hgnc',
            'HOGENOM': 'xref_hogenom',
            'HPA': 'xref_hpa',
            'IDEAL': 'xref_ideal',
            'IMGT_GENE-DB': 'xref_imgt_gene-db',
            'InParanoid': 'xref_inparanoid',
            'IntAct': 'xref_intact',
            'InterPro': 'xref_interpro',
            'iPTMnet': 'xref_iptmnet',
            'jPOST': 'xref_jpost',
            'KEGG': 'xref_kegg',
            'KO': 'xref_ko',
            'LegioList': 'xref_legiolist',
            'Leproma': 'xref_leproma',
            'MaizeGDB': 'xref_maizegdb',
            'MalaCards': 'xref_malacards',
            'MANE-Select': 'xref_mane-select',
            'MassIVE': 'xref_massive',
            'MaxQB': 'xref_maxqb',
            'MEROPS': 'xref_merops',
            'MetOSite': 'xref_metosite',
            'MGI': 'xref_mgi',
            'MIM': 'xref_mim',
            'MINT': 'xref_mint',
            'MoonDB': 'xref_moondb',
            'MoonProt': 'xref_moonprot',
            'neXtProt': 'xref_nextprot',
            'NIAGADS': 'xref_niagads',
            'OGP': 'xref_ogp',
            'OMA': 'xref_oma',
            'OpenTargets': 'xref_opentargets',
            'Orphanet': 'xref_orphanet',
            'OrthoDB': 'xref_orthodb',
            'PANTHER': 'xref_panther',
            'PathwayCommons': 'xref_pathwaycommons',
            'PATRIC': 'xref_patric',
            'PaxDb': 'xref_paxdb',
            'PCDDB': 'xref_pcddb',
            'PDB': 'xref_pdb',
            'PDBsum': 'xref_pdbsum',
            'PeptideAtlas': 'xref_peptideatlas',
            'PeroxiBase': 'xref_peroxibase',
            'Pfam': 'xref_pfam',
            'PharmGKB': 'xref_pharmgkb',
            'Pharos': 'xref_pharos',
            'PHI-base': 'xref_phi-base',
            'PhosphoSitePlus': 'xref_phosphositeplus',
            'PhylomeDB': 'xref_phylomedb',
            'PIR': 'xref_pir',
            'PIRSF': 'xref_pirsf',
            'PlantReactome': 'xref_plantreactome',
            'PomBase': 'xref_pombase',
            'PRIDE': 'xref_pride',
            'PRINTS': 'xref_prints',
            'PRO': 'xref_pro',
            'ProDom': 'xref_prodom',
            'ProMEX': 'xref_promex',
            'PROSITE': 'xref_prosite',
            'Proteomes': 'xref_proteomes',
            'ProteomicsDB': 'xref_proteomicsdb',
            'PseudoCAP': 'xref_pseudocap',
            'Reactome': 'xref_reactome',
            'REBASE': 'xref_rebase',
            'RefSeq': 'xref_refseq',
            'REPRODUCTION-2DPAGE': 'xref_reproduction-2dpage',
            'RGD': 'xref_rgd',
            'RNAct': 'xref_rnact',
            'SABIO-RK': 'xref_sabio-rk',
            'SASBDB': 'xref_sasbdb',
            'SFLD': 'xref_sfld',
            'SGD': 'xref_sgd',
            'SignaLink': 'xref_signalink',
            'SIGNOR': 'xref_signor',
            'SMART': 'xref_smart',
            'SMR': 'xref_smr',
            'STRING': 'xref_string',
            'SUPFAM': 'xref_supfam',
            'SWISS-2DPAGE': 'xref_swiss-2dpage',
            'SwissLipids': 'xref_swisslipids',
            'SwissPalm': 'xref_swisspalm',
            'TAIR': 'xref_tair',
            'TCDB': 'xref_tcdb',
            'TIGRFAMs': 'xref_tigrfams',
            'TopDownProteomics': 'xref_topdownproteomics',
            'TreeFam': 'xref_treefam',
            'TubercuList': 'xref_tuberculist',
            'UCD-2DPAGE': 'xref_ucd-2dpage',
            'UCSC': 'xref_ucsc',
            'UniCarbKB': 'xref_unicarbkb',
            'UniLectin': 'xref_unilectin',
            'UniPathway': 'xref_unipathway',
            'VectorBase': 'xref_vectorbase',
            'VEuPathDB': 'xref_veupathdb',
            'VGNC': 'xref_vgnc',
            'WBParaSite': 'xref_wbparasite',
            'WBParaSiteTranscriptProtein': 'xref_wbparasitetranscriptprotein',
            'World-2DPAGE': 'xref_world-2dpage',
            'WormBase': 'xref_wormbase',
            'Xenbase': 'xref_xenbase',
            'ZFIN': 'xref_zfin'
            }
fils = {v:k for k,v in fils.items()}
for f in [
    "accession",
    "reviewed",
    "gene_primary",
    "id",
    "gene_names",
    "organism_name",
    "length",
    "cc_alternative_products",
    "ft_var_seq",
    "sequence"
]:
    print(fils[f])

Entry
Reviewed
Gene names (primary)
Entry name
Gene names
Organism
Length
Alternative products
Alternative sequence
Sequence


In [34]:
len(data['results'])

50

In [11]:
import requests
import json

# Base URL for UniProt API
UNIPROT_API_URL = "https://rest.uniprot.org/taxonomy"

# Parameters for the API request
params = {
    "fields": "name,common_name,synonym,scientific_name,rank",
    "format": "json",
    "size": 500,
}

def fetch_organisms():
    """Fetch all organisms from UniProt API and save them to a JSON file."""
    all_organisms = []
    url = UNIPROT_API_URL
    
    while url:
        print(f"Fetching: {url}")
        response = requests.get(url, params=params if url == UNIPROT_API_URL else None)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break
        
        data = response.json()
        all_organisms.extend(data.get("results", []))
        url = data.get("next")  # URL for the next page

    print(f"Fetched {len(all_organisms)} organisms.")
    return all_organisms

if __name__ == "__main__":
    fetch_organisms()


Fetching: https://rest.uniprot.org/taxonomy
Error: 404 - <html>
<head><title>404 Not Found</title></head>
<body>
<center><h1>404 Not Found</h1></center>
<hr><center>nginx/1.25.3</center>
</body>
</html>

Fetched 0 organisms.


In [7]:
len(df['Organism'].unique())

14456

In [5]:
df.columns

Index(['Entry', 'Reviewed', 'Gene Names (primary)', 'Entry Name', 'Gene Names',
       'Organism', 'Length', 'Alternative products (isoforms)',
       'Alternative sequence', 'Sequence'],
      dtype='object')

In [11]:

db_file = os.path.join('data','db','proteogyver.db')
db_conn = db_functions.create_connection(db_file)
new_current = db_functions.get_full_table_as_pd(db_conn, 'common_proteins')
db_conn.close()

In [9]:
to_modify = []
for _,row in current[current['protein_type'].str.contains('ipynb')].iterrows():
    to_modify.append([row['uniprot_id'], row['protein_type'].replace('.ipynb_checkpoints, ', '')])
db_conn = db_functions.create_connection(db_file)
for mod in to_modify:
    db_functions.modify_record(db_conn, 'common_proteins', 'uniprot_id', mod[0], ['protein_type'], [mod[1]])
db_conn.commit()
db_conn.close()

In [15]:
new_current['protein_type'].value_counts()

protein_type
Proteasome                                           474934
Keratin                                               75348
Elongation factor                                      5290
Ribosome                                               3942
Carboxylase                                            2816
Chaperone                                              2351
Histone                                                2017
Heat shock protein                                     1373
Actin                                                  1207
Chaperone, Heat shock protein                          1135
Tubulin                                                 671
Heat shock protein, Proteasome                          117
Chaperone, Histone                                       89
Keratin, Proteasome                                      71
Elongation factor, Ribosome                              40
Chaperone, Proteasome                                    40
Chaperone, Tubulin         

In [14]:
current['protein_type'].value_counts().sum()

571622

In [4]:
current.head()

,uniprot_id,gene_name,entry_name,all_gene_names,organism,protein_type,update_time
0,P04792,HSPB1,HSPB1_HUMAN,HSPB1 HSP27 HSP28,Homo sapiens (Human),"Heat shock protein, Proteasome",2024-11-21
1,P63284,clpB,CLPB_ECOLI,clpB htpM b2592 JW2573,Escherichia coli (strain K12),"Chaperone, Heat shock protein",2024-11-21
2,P10591,SSA1,HSP71_YEAST,SSA1 YAL005C,Saccharomyces cerevisiae (strain ATCC 204508 /...,"Heat shock protein, Proteasome",2024-11-21
3,Q00613,HSF1,HSF1_HUMAN,HSF1 HSTF1,Homo sapiens (Human),"Heat shock protein, Proteasome",2024-11-21
4,O14558,HSPB6,HSPB6_HUMAN,HSPB6,Homo sapiens (Human),Heat shock protein,2024-11-21


In [6]:
current[current['gene_name'].fillna('').str.contains('ATP')]

,uniprot_id,gene_name,entry_name,all_gene_names,organism,protein_type,update_time
1372,P27449,ATP6V0C,VATL_HUMAN,ATP6V0C ATP6C ATP6L ATPL,Homo sapiens (Human),Proteasome components,2024-02-16
3407,Q9NQ11,ATP13A2,AT132_HUMAN,ATP13A2 PARK9,Homo sapiens (Human),Proteasome components,2024-02-16
3966,P23956,ATP6V0C,VATL_BOVIN,ATP6V0C ATP6C ATP6L,Bos taurus (Bovine),Proteasome components,2024-02-16
5297,O18882,ATP6V0C,VATL_SHEEP,ATP6V0C ATP6C ATP6L,Ovis aries (Sheep),Proteasome components,2024-02-16
7747,Q9LU63,ATP1,ATP1_ARATH,ATP1 PDL1 SDIRIP1 At5g51110 MWD22.5,Arabidopsis thaliana (Mouse-ear cress),Carboxylases,2024-02-16


In [ ]:
comdir = os.path.join('data_assets','db build files','Potential contaminant protein groups')
comdir = '/home/kmsaloka/Downloads/Database additions/Potential contaminant protein groups'
db_file = os.path.join('data','proteogyver.db')

com_cols = [
    'uniprot_id TEXT PRIMARY KEY',
    'gene_name TEXT',
    'entry_name TEXT',
    'all_gene_names TEXT',
    'organism TEXT',
    'protein_type TEXT NOT NULL',
    'update_time TEXT NOT NULL '
]
db_conn = db_functions.create_connection(db_file)
current = db_functions.get_full_table_as_pd(db_conn, 'common_proteins')
have = set(current['uniprot_id'].values)
db_conn.close()
common_proteins = {}
for root, dirs, files in os.walk(comdir):
    for f in files:
        comdf = pd.read_csv(os.path.join(root,f),sep='\t')
        name = root.rsplit(os.sep,maxsplit=1)[-1]
        for _,row in comdf.iterrows():
            if row['Entry'] not in common_proteins:
                common_proteins[row['Entry']] = [
                    row['Entry'],
                    row['Gene Names (primary)'],
                    row['Entry Name'],
                    row['Gene Names'],
                    row['Organism'],
                    [name],
                    datetime.today().strftime('%Y-%m-%d')
                ]
            else:
                common_proteins[row['Entry']][5].append(name)

In [ ]:
modifications = []
additions = []
for common_protein, data in common_proteins.items():
    if common_protein in have:
        cur_ver = current[current['uniprot_id']==common_protein]
        mods = cur_ver['protein_type'].split(', ')
        mods.extend(data[5])
        modifications.append([common_protein, 'protein_type', ', '.join(sorted(list(set(mods))))])
    else:
        data[5] = ', '.join(data[5])
        additions.append([com_cols, data])
db_conn = db_functions.create_connection(db_file)
for addcols, add_data in additions:
    db_functions.add_record(db_conn, 'common_proteins', addcols, add_data)

In [12]:
current.head()

,uniprot_id,gene_name,entry_name,all_gene_names,organism,protein,update_time
0,A0A0G2JZ79,Sirt1,SIR1_RAT,Sirt1,Rattus norvegicus (Rat),Keratin,2024-02-05
1,A0A0H2URK1,psrP,PSRP_STRPN,psrP SP_1772,Streptococcus pneumoniae serotype 4 (strain AT...,Keratin,2024-02-05
2,D3ZER2,Bfsp2,BFSP2_RAT,Bfsp2,Rattus norvegicus (Rat),Keratin,2024-02-05
3,F7EQ49,GPER1,GPER1_MACMU,GPER1 GPER GPR30,Macaca mulatta (Rhesus macaque),Keratin,2024-02-05
4,O08878,Gper1,GPER1_RAT,Gper1 Cmkrl2 Gper Gpr30 Gpr41,Rattus norvegicus (Rat),Keratin,2024-02-05


In [21]:
def rename_column(db_conn, tablename, old_col, new_col):
    sql_str = f'ALTER TABLE {tablename} RENAME COLUMN {old_col} TO {new_col};'
    try:
        db_conn.cursor().execute(sql_str)
    except sqlite3.Error as error:
        print(f'Failed to rename column in sqlite table. Error: {error}', sql_str)
        raise
db_file = os.path.join('data','proteogyvertesting.db')
con = db_functions.create_connection(db_file)
rename_column(con, 'common_proteins','protein','protein_type')
con.commit()
con.close()

In [20]:
import sqlite3
con = sqlite3.connect(db_file)
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
con.close()

[('control_sets',), ('crapome_sets',), ('proteins',), ('control_vl_gfp_mac3_10min_ap_overall',), ('control_vl_gfp_mac3_10min_ap',), ('control_vl_gfp_mac3_10min_bioid_overall',), ('control_vl_gfp_mac3_10min_bioid',), ('control_vl_gfp_mac2_18h_ap_overall',), ('control_vl_gfp_mac2_18h_ap',), ('control_vl_gfp_mac2_18h_bioid_overall',), ('control_vl_gfp_mac2_18h_bioid',), ('control_vl_gfp_mac_24h_ap_overall',), ('control_vl_gfp_mac_24h_ap',), ('control_vl_gfp_mac_24h_ap_nls_overall',), ('control_vl_gfp_mac_24h_ap_nls',), ('control_vl_gfp_mac_24h_bioid_overall',), ('control_vl_gfp_mac_24h_bioid',), ('control_vl_gfp_mac_24h_bioid_nls_overall',), ('control_vl_gfp_mac_24h_bioid_nls',), ('crapome_vl_gfp_mac3_10min_ap',), ('crapome_vl_gfp_mac3_10min_bioid',), ('crapome_vl_gfp_mac2_18h_ap',), ('crapome_vl_gfp_mac2_18h_bioid',), ('crapome_vl_gfp_mac_24h_ap',), ('crapome_vl_gfp_mac_24h_ap_nls',), ('crapome_vl_gfp_mac_24h_bioid',), ('crapome_vl_gfp_mac_24h_bioid_nls',), ('crapome_nesvilab',), ('conta